# Miller-Rabin Primality Test

Let's recap a few results from the notebook "PRIMES is in NP":
* First, we noted that the traditional algorithm for determining whether a number $n$ is prime (consisting in trying all potential divisors up to $\sqrt n$) is very slow: it runs in exponential time in the number of digits of $n$.
* We then tried to use __Fermat's Little Theorem__ as a criteria of primality: if $n$ is prime and $a \not\equiv_n 0$ then $a^{n-1} \equiv_n 1$.
* Unfortutely, its converse is not true. For example $2^{340} \equiv_{341} 1$ and $3^{90} \equiv_{91} 1$, even though $341$ and $91$ are composite. There does not appear to be a witness $a$ that would work with every composite.
* We finally tried using fixed small sets of witnesses $\{a_1, a_2, \ldots, a_k\}$, but gave up as they don't appear to work for every composite either.

In the present notebook, we show first that for most composites, by choosing a __random__ small set of witnesses, Fermat's test provides a proof of compositeness with very high probability.

A few composites, called __Carmichael Numbers__, fail this test. Fortunately, they can be proven composite too, in a randomized sense again, by using a stronger version of Fermat's test, called __Miller-Rabin test__.

## Algorithm


The algorithm runs first Fermat's test. If it is unable to determine that $n$ is composite, then it means that $a^{n-1} \equiv_n 1$. In this case, the algorithm computes the square root of $a^{n-1}$, that is $a^{{n-1} \over 2}$. If the result is $\not\equiv_n \pm 1$, it means that $n$ has a nontrivial square root of $1$ and therefore it is composite. If it is $1$, the algorithm tries to find a nontrivial square root of $n$ again etc.

In [1]:
from random import *

# Tries to determine whether n is prime by running witness_test 
# on k random potential witnesses.
# Returns false if any of these witnesses determines n is composite.
# Returns true otherwise.
def is_probable_prime(n, witness_test, k = 50):
    for i in range(k):
        a = randint(1, n - 1)
        if witness_test(a, n):
            return False
    return True

# Returns true if Miller-Rabin witness test shows that n
# is composite using the witness a.
def is_miller_rabin_witness(a, n):
    if pow(a, n - 1, n) != 1:
        return True
    k = n - 1
    while k % 2 == 0:
        k //= 2
        res = pow(a, k, n)
        if res == -1 + n:
            return False
        if res != 1:
            # Found square root of 1 different from +/- 1.
            return True
    return False

def is_miller_rabin_probable_prime(n):
    return is_probable_prime(n, is_miller_rabin_witness)

__Notes__

* If the input is prime, the test is always correct.
* If, for a given composite $n$, the number of witnesses between $1$ and $n-1$ is $w$, then the test will succeed with probability $1-\left({{n-1-w}\over{n-1}}\right)^k$.
* We will show later that for Miller-Rabin at least half of these numbers are witnesses. Therefore, for $k=50$, the probability of success is at least $1 - \left({1 \over 2}\right)^{50}$.
* Our version is slightly different from the classic version of the algorithm. It may be a little slower in some cases, but it is a little simpler and still runs in polynomial time.

__Examples__

What is remarkable is that before Miller-Rabin (and Solovay-Strassen), there wasn't a way to be confident that a big number, say with 100 digits, was prime, except for very special cases such as Mersenne Primes.

Miller-Rabin changes that. Now we can determine in less than a second the first 100-digit prime:

In [2]:
d = 100
print("The first ", d,"-digit prime is...\n", sep = '')
n = 10**(d-1)
while not is_miller_rabin_probable_prime(n):
    n += 1
print(n)

The first 100-digit prime is...

1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000289


We can also determine primes with only 1's.

In [3]:
for d in range(2, 500):
    n = (10**d - 1)//9
    if is_miller_rabin_probable_prime(n):
        print(n)
        print()

11

1111111111111111111

11111111111111111111111

11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111



Primes that are the concatenation of primes:

In [4]:
n = 0
for p in range(2, 1000):
    if is_miller_rabin_probable_prime(p):
        if p < 10:
            n = n*10 + p
        elif p < 100:
            n = n*100 + p
        elif p < 1000:
            n = n*1000 + p
        else:
            n = n*10000 + p
        if is_miller_rabin_probable_prime(n):
            print(n)
            print()

2

23

2357

2357111317192329313741434753596167717379838997101103107109113127131137139149151157163167173179181191193197199211223227229233239241251257263269271277281283293307311313317331337347349353359367373379383389397401409419421431433439443449457461463467479487491499503509521523541547557563569571577587593599601607613617619631641643647653659661673677683691701709719



## Other Primality Tests

In this notebook, witnesses will refer to witnesses of __compositeness__ of a number $n$. A number $a$ 
is a witness of compositeness of $n$, according to some witness function, if running the function on input $a$ and $n$ returns true.

In general the more witnesses there are for $n$, the better the witness function.

There are other witness functions which are simpler than Miller-Rabin, but they are not as good. They rank as follows:

$$\mathrm{Divisor < GCD < Fermat < Euler < Miller-Rabin}$$

### Divisor trial test

This is the simple test used in the traditional primality algorithm. The witnesses are the nontrivial divisors of $n$. 

This is a very weak test. For example if $n$ is the product of 2 prime numbers, there are __only $2$ divisor witnesses__.

In [5]:
# In all these tests, we assume n >= 1 and 0 < a < n.

def is_div_witness(a, n):
    if a == 1: return False
    return n % a == 0

### GCD test

This test is better than the previous one, but it is still weak.

For example when $n$ is the product of 2 large prime numbers $n=pq$, the test fails if $(a, n) = 1$. If $\phi(n)$ denotes Euler's totient function, there are $\phi(n)$ non-witnesses, and $n - 1 - \phi(n)$ witnesses.

Since $\phi(n) = n(1-{1\over p})(1-{1\over q})$, there are __only $p + q - 2$ GCD witnesses__, which is very small compared to $n-1$.

In [6]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

def is_gcd_witness(a, n):
    return gcd(a, n) > 1

### Fermat's test

This test is based on Fermat's little theorem. It is much better than the 2 previous ones. For __most__ composites, but not all, there are __at least ${n-1}\over 2$ Fermat witnesses__.

In [7]:
def is_fermat_witness(a, n):
    return pow(a, n - 1, n) != 1

### Euler's test

This test is better than Fermat's but as not as good as Miller-Rabin. For __all__ composites, there at __at least ${n-1}\over 2$ Miller-Rabin witnesses__, but this is not the case for Euler witnesses.

In [8]:
def is_euler_witness(a, n):
    if n % 2 == 0: return False
    b = pow(a, (n - 1) // 2, n)
    return b != 1 and b != -1 + n 

## Comparing Primality Tests

Let's count the proportion of witnesses for some composites.

In [9]:
# Counts the number of witnesses between 1 and n - 1.
def count_witnesses(n, is_witness):
    count = 0
    for a in range(1, n):
        if is_witness(a, n):
            count += 1
    return count

# Returns True if n is prime.
def is_prime(n):
    if n <= 1: return False
    if n == 2: return True
    if n % 2 == 0: return False
    i = 3
    while (i*i <= n):
        if n % i == 0:
            return False
        i += 2
    return True

def count_witnesses_for_list(list, counting_method = count_witnesses):
    print('Percentage of witnesses for n:')
    print()
    print('              n      divisor      gcd       fermat     euler   miller-rabin')
    print()
    for n in list:
        if n % 2 == 0 or is_prime(n):
            continue
        print('%15d' % n, end = ' ')
        for is_witness in [is_div_witness, 
                           is_gcd_witness, 
                           is_fermat_witness, 
                           is_euler_witness, 
                           is_miller_rabin_witness]:
            count = counting_method(n, is_witness)
            print('%10d' % (count*100//(n-1)), end = ' ')
        print()

We exclude prime numbers, which obviously do not have any witnesses for these tests. We exclude even numbers since they aren't handled by Euler's test. They are obviously composite anyway.

__Composites up to 100__

Note that the divisor test is poor even for small numbers.

In [10]:
count_witnesses_for_list(range(2, 101))

Percentage of witnesses for n:

              n      divisor      gcd       fermat     euler   miller-rabin

              9         12         25         75         75         75 
             15         14         42         71         85         85 
             21         10         40         80         80         90 
             25          4         16         83         83         83 
             27          7         30         92         92         92 
             33          6         37         87         87         93 
             35          5         29         88         94         94 
             39          5         36         89         94         94 
             45          9         45         81         90         95 
             49          2         12         87         87         87 
             51          4         36         92         96         96 
             55          3         25         92         96         96 
             57          3 

__Composites from 10000 to 10100__

For these larger numbers, divisor test is very poor. GCD test is poor for several numbers. Other tests are good for these numbers.

In [11]:
count_witnesses_for_list(range(10000, 10100))

Percentage of witnesses for n:

              n      divisor      gcd       fermat     euler   miller-rabin

          10001          0          2         99         99         99 
          10003          0         14         99         99         99 
          10005          0         50         99         99         99 
          10011          0         35         97         99         99 
          10013          0         13         99         99         99 
          10015          0         20         99         99         99 
          10017          0         43         99         99         99 
          10019          0          2         99         99         99 
          10021          0          9         99         99         99 
          10023          0         38         99         99         99 
          10025          0         20         99         99         99 
          10027          0          3         96         98         98 
          10029          0 

## Carmichael Numbers

__Small Carmichael Numbers__

So far, we know that the divisor and gcd tests are not good enough. Other tests looks pretty good. Let's look more in detail at Fermat's test. Does it ever do worse than 50%?

Let's find out:

In [12]:
charmichael_numbers = []
for n in range(2, 3000):
    if n % 2 == 0 or is_prime(n):
        continue
    count = count_witnesses(n, is_fermat_witness)
    if count < (n - 1) // 2:
        charmichael_numbers.append(n)

count_witnesses_for_list(charmichael_numbers)

Percentage of witnesses for n:

              n      divisor      gcd       fermat     euler   miller-rabin

            561          1         42         42         71         98 
           1105          0         30         30         65         97 
           1729          0         25         25         25         90 
           2465          0         27         27         27         97 
           2821          0         23         23         61         90 


So indeed, there are composites for which Fermat doesn't do as well. In fact, for these composites, called Carmichael numbers, the only witnesses are the trivial ones, the ones that have a factor in common with $n$. This is why the "gcd" and "fermat" column are identical.

__651693055693681__

For some Carmichael numbers, Fermat and Euler are really bad. For example for $n = 651693055693681$.

Since it's a large number, we will only estimate the proportion of composites.

In [13]:
def estimate_witnesses(n, is_witness):
    count = 0
    for k in range(0, 10000):
        if is_witness(randint(1, n-1), n):
            count += 1
    return count * (n - 1) / 10000

count_witnesses_for_list([651693055693681], estimate_witnesses)

Percentage of witnesses for n:

              n      divisor      gcd       fermat     euler   miller-rabin

651693055693681          0          0          0          0         87 


In fact, $n = 651693055693680$ is in very special number. It is the product of $3$ prime numbers $p$, $q$ and $r$ such that:
* $p-1 \mid n-1$
* $q-1 \mid n-1$
* $r-1 \mid n-1$

In [14]:
n = 651693055693681
p = 72931
q = 87517
r = 102103

assert(n == p*q*r)
assert((n - 1) % (p - 1) == 0)
assert((n - 1) % (q - 1) == 0)
assert((n - 1) % (r - 1) == 0)

For such numbers $(a, p) = 1$ implies $a^{n-1} \equiv_p 1$, therefore they only have $n - 1 - \phi(n)$ witnesses.

__Proof__

Since $p$ is prime, if $(a, p) = 1$, $a^{p-1} \equiv_p 1$. And since $p-1 \mid n-1$: $a^{n-1} \equiv_p 1$.

Similarly:
* if $(a, q) = 1$, $a^{n-1} \equiv_q 1$
* if $(a, r) = 1$, $a^{n-1} \equiv_r 1$

This means, using the Chinese Remainder Theorem, that if $(a, n) = 1$, $a^{n-1} \equiv_n 1$.

That is, even though $n$ is composite, it is pseudoprime in $\phi(n)$ basis. Since: 

$$\phi(n) = n\left(1-{1 \over p}\right)\left(1-{1 \over q}\right)\left(1-{1 \over r}\right)$$

and $p$, $q$ and $r$ are large primes, ${\phi(n)} \over n$ is close to $1$.

__Chernick's Carmichael numbers__

Chernick noted, and this is easy to verify, that if $6k + 1$, $12k + 1$ and $18k + 1$ are prime then the product is a Carmichael number.

Let's try to find a few such numbers:

In [15]:
chernick_numbers = []

for k in range(0, 500):
    p = 6*k + 1
    q = 12*k + 1
    r = 18*k + 1
    if is_prime(p) and is_prime(q) and is_prime(r):
        chernick_numbers.append(p * q * r)

count_witnesses_for_list(chernick_numbers, estimate_witnesses)

Percentage of witnesses for n:

              n      divisor      gcd       fermat     euler   miller-rabin

           1729          0         24         24         24         90 
         294409          0          4          5         53         92 
       56052361          0          0          0          0         87 
      118901521          0          0          0          0         87 
      172947529          0          0          0          0         87 
      216821881          0          0          0          0         87 
      228842209          0          0          0         49         93 
     1299963601          0          0          0         49         93 
     2301745249          0          0          0          0         87 
     9624742921          0          0          0          0         87 
    11346205609          0          0          0         50         92 
    13079177569          0          0          0         50         93 
    21515221081          0 

## Proof of Miller-Rabin

So far, from our experiments, we know that, of the 5 tests, only Miller-Rabin does consistently well.

In fact, we will prove that it does well on __any__ composite.

First a lemma.

__Lemma:__ Carmichael numbers are divisible by 2 different primes.

__Proof:__ If $n$ is prime, obviously it cannot be a Carmichael number. Let's prove that $n$ is not a power of prime $p$: $n \neq p^k$, with $k \geq 2$.

Since $Z^*_{p^k}$ is cyclic, it has a generator $g$. Therefore $(g^1, g^2, \ldots, g^i, \ldots )$ is a sequence of period $\phi(p^k)$, with $g^{\phi(p^k)} = 1$. Since $\phi(p^k) = p^k - p$, we have $\phi(p^k) < p^k - 1 < 2\phi(p^k)$, which means $g^{n - 1} \not\equiv_n 1$.

__Theorem:__ if $n$ is composite, there are at least ${n-1}\over{2}$ Miller-Rabin witnesses.

__Proof:__ For each $a$ in $Z^*_n$, consider the sequence that goes on while $2^k \mid n-1$.

$$s(a) = (a^{{n-1} \over 1}, a^{{n-1} \over 2}, a^{{n-1} \over 4}, \ldots )$$

We can partition $Z^*_n$, according to how that sequence looks like: 
* $W_{\alpha}(n)$: $\{\alpha, \ldots\}$, $\alpha \not\equiv_n 1$ (Fermat witness) 
* $W_{\beta}(n)$: $\{1, 1, \ldots, 1, \beta, \ldots\}$, $\beta \not\equiv \pm 1$ (nontrivial square root of $1$).
* $L_{-1}(M)$: $\{1, 1, \ldots, 1, -1, \ldots \}$.
* $L_1(M)$: $\{1, 1, \ldots, 1\}$ 

$$Z^*_n = W_\alpha(n) \cup W_\beta(n) \cup L_1(n) \cup L_{-1}(n)$$ 

Let's now prove the theorem, by distinguishing 2 cases.

__case 1: n is not a Carmichael number__

It's easy to see that $W_\beta(n) \cup L_1(n) \cup L_{-1}(n)$ is a subgroup of $Z^*_n$. Since $W_{\alpha}(n)$ is not empty, the subgroup is proper. It has at most ${\phi(n)} \over 2$ elements, since the order of a subgroup divides the order of the group (Lagrange's theorem).

That is, there at most ${{n - 1} \over {2}}$ liars, or equivalently at least ${{n - 1} \over {2}}$ witnesses.

__case 2: n is a Carmichael number__

Note that $-1 \in L_{-1}(n)$ and therefore $L_{-1}(n)$ is not empty. Among all the elements of $L_{-1}(n)$, consider $u \in L_{-1}(n)$ such that the index $i$ for the first $-1$ in $s(u)$ is minimized.

Consider the set $B$ of numbers such that their sequence starts with $i-1$ 1's followed by $\pm 1$.

Clearly $L_{1}(n) \cup L_{-1}(n) \subseteq B$ and $B$ is a subgroup of $Z^*_n$.

So, following the same reasoning as before, it's enough to show that $Z^*_n - B$ has at least 1 element.

We have $u^{{n-1} \over {2^i}} \equiv_n -1$

Let's write $n = n_1n_2$, with $(n_1,n_2)=1$. This is possible according to the previous lemma.

Then $u^{{n-1} \over {2^i}} \equiv_{n_1} -1$ and $u^{{n-1} \over {2^i}} \equiv_{n_2} -1$.

Consider now the unique $v$ such that $v \equiv_{n_1} 1$ and $v \equiv_{n_2} u$.

Then $v^{{n-1} \over {2^i}} \not\equiv_n \pm 1$ and $v^{{n-1} \over {2^{i-1}}} \equiv_n 1$. Therefore $v \in Z^*_n - B$.
